In [46]:
import requests
import lxml.html as lh
import pandas as pd

## Récupération des données par web Scraping

Récupération d'une liste de jeu simple avec nom, desription et date de sortie : http://www.jeuxvideo.com/meilleurs/machine-20/

### Variables globales

In [47]:
# URL
url = 'http://www.jeuxvideo.com/meilleurs/'
# URI-SUFFIX
uri_pages = '?p='
# Number of pages to read
nbPages = 10
# XPath content to collect
tags = ['//a[@class="gameTitleLink__196nPy"]', \
        '//p[@class="description__1-Pqha"]' , \
        '//span[@class="releaseDate__1RvUmc"]', \
        '//span[@class="editorialRating__1tYu_r"]', \
        '//span[@class="userRating__1y96su"]']

In [48]:
page = requests.get(url)
doc = lh.fromstring(page.content)

On récupère les données via des requêtes XPath

In [49]:
nomJeux = doc.xpath('//a[@class="gameTitleLink__196nPy"]')
desc = doc.xpath('//p[@class="description__1-Pqha"]')
sortie = doc.xpath('//span[@class="releaseDate__1RvUmc"]')
test = doc.xpath('//span[@class="editorialRating__1tYu_r"]')
avis = doc.xpath('//span[@class="userRating__1y96su"]')

Affichage des données décomposées :

In [50]:
for i in range(len(nomJeux)):
    print(nomJeux[i].text_content().strip() + "\n" + \
          desc[i].text_content().strip() + "\n" + \
          sortie[i].text_content().strip()+ "\n" + \
          test[i].text_content().strip() + "\n" + \
          avis[i].text_content().strip() + "\n")

God of War sur PS4
Dans ce nouvel épisode de God Of War, le héros évoluera dans un monde aux inspirations nordiques, très forestier et montagneux. Dans ce beat-them-all, un enfant accompagnera le héros principal, pouvant apprendre des actions du joueur, et même gagner de l'expérience.
Sortie: 20 avril 2018
20/20
17.7/20

The Legend of Zelda : Breath of the Wild
The Legend of Zelda : Breath of the Wild est un jeu d'action/aventure. Link se réveille d'un sommeil de 100 ans dans un royaume d'Hyrule dévasté. Il lui faudra percer les mystères du passé et vaincre Ganon, le fléau. L'aventure prend place dans un gigantesque monde ouvert et accorde ainsi une part importante à l'exploration. Le titre a été pensé pour que le joueur puisse aller où il veut dès le début, s'éloignant de la linéarité habituelle de la série.
Sortie: 3 mars 2017
20/20
17.5/20

The Legend of Zelda : The Wind Waker sur NGC
Dans The Legend of Zelda : The Wind Waker, vous incarnez Link dans sa quête pour sauver sa sœur. Dé

IndexError: list index out of range

### Mettons les données dans un DataFrame Pandas

On décortique volontairement ici le cheminement ...

In [ ]:
l1 = pd.DataFrame([nomJeux[i].text_content().strip() for i in range(len(nomJeux))], columns=['Jeux'])
l1['key'] = l1.index
l2 = pd.DataFrame([desc[i].text_content().strip() for i in range(len(desc))], columns=['Description'])
l2['key'] = l2.index
l3 = pd.DataFrame([sortie[i].text_content().strip().replace("Sortie: ", "") for i in range(len(sortie))], columns=['Sortie'])
l3['key'] = l3.index
l4 = pd.DataFrame([test[i].text_content().strip() for i in range(len(test))], columns=['Test'])
l4['key'] = l4.index
l5 = pd.DataFrame([avis[i].text_content().strip() for i in range(len(avis))], columns=['Avis'])
l5['key'] = l5.index

liste = l1.join(l2, on='key', how='left', lsuffix='_l1', rsuffix='_r1').join(l3, on='key_l1', how='left', lsuffix='_l1', rsuffix='_r2')
del liste['key_l1']
del liste['key_r1']
liste = liste.join(l4, on='key', how='left', lsuffix='_l1', rsuffix='_r1').join(l5, on='key_l1', how='left', lsuffix='_l1', rsuffix='_r2')
del liste['key_l1']
del liste['key_r1']
del liste['key']
liste.head()

## Création de la fonction de récupération des données d'une page

In [ ]:
def getPage(url):
    page = requests.get(url)
    doc = lh.fromstring(page.content)

    # Get the Web data via XPath
    content = []
    for i in range(len(tags)):
        content.append(doc.xpath(tags[i]))

    # Gather the data into a Pandas DataFrame array
    df_liste = []
    for j in range(len(tags)):
        tmp = pd.DataFrame([content[j][i].text_content().strip() for i in range(len(content[i]))], columns=[tags[j]])
        tmp['key'] = tmp.index
        df_liste.append(tmp)

    # Build the unique Dataframe with one tag (xpath) content per column
    liste = df_liste[0]
    for j in range(len(tags)-1):
        liste = liste.join(df_liste[j+1], on='key', how='left', lsuffix='_l', rsuffix='_r')
        liste['key'] = liste.index
        del liste['key_l']
        del liste['key_r']
    
    return liste

In [ ]:
getPage(url).head()

## Gestion de la pagination

In [ ]:
def getPages(_nbPages, _url):
    liste_finale = pd.DataFrame()
    for i in range (_nbPages):
        liste = getPage(_url + uri_pages + str(i+1))
        liste_finale = pd.concat([liste_finale, liste], ignore_index=True)
    return liste_finale

liste_totale = getPages(nbPages, url)
liste_totale.head()

Ecrit la liste dans un fichier

In [76]:
liste_totale.to_csv('meilleursjeuvideo.csv', index=False)